In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn, optim
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import copy


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:


def creat_data(n):
    data = pd.read_csv("Heart Attack.csv")
    data_start = pd.read_csv("Heart Attack.csv")
    df_chunks = np.array_split(data_start, n)

    trainloaders = []
    testloaders = []

    X = data.drop(columns='class')
    y = data['class']

    X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.1, random_state=0)

    X_train1 = torch.Tensor(X_train1.values)
    y_train1 = torch.Tensor(y_train1.values)
    X_train1 = X_train1.to(dtype=torch.float32).clone().detach()
    y_train1 = y_train1.to(dtype=torch.float32).reshape(-1, 1).clone().detach()

    X_train1 = X_train1.to(device)
    y_train1 = y_train1.to(device)

    X_test1 = torch.Tensor(X_test1.values)
    y_test1 = torch.Tensor(y_test1.values)
    X_test1 = X_test1.to(dtype=torch.float32).clone().detach()
    y_test1 = y_test1.to(dtype=torch.float32).reshape(-1, 1).clone().detach()

    X_test1 = X_test1.to(device)
    y_test1 = y_test1.to(device)

    for data in df_chunks:
        hold1 = []
        hold2 = []

        X = data.drop(columns='class')
        y = data['class']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

        X_train = torch.Tensor(X_train.values)
        y_train = torch.Tensor(y_train.values)
        X_train = X_train.to(dtype=torch.float32).clone().detach()
        y_train = y_train.to(dtype=torch.float32).reshape(-1, 1).clone().detach()

        X_train = X_train.to(device)
        y_train = y_train.to(device)

        X_test = torch.Tensor(X_test.values)
        y_test = torch.Tensor(y_test.values)
        X_test = X_test.to(dtype=torch.float32).clone().detach()
        y_test = y_test.to(dtype=torch.float32).reshape(-1, 1).clone().detach()

        X_test = X_test.to(device)
        y_test = y_test.to(device)

        hold1.append((X_train, y_train))
        hold2.append((X_test, y_test))
        trainloaders.append(hold1)
        testloaders.append(hold2)

    return trainloaders, testloaders, X_train1, y_train1

In [ ]:
# model definition
class ModelA(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(8, 8)    
        self.hidden2 = nn.Linear(8, 16)
        self.act2 = nn.Sigmoid()
        self.hidden3 = nn.Linear(16, 32)
        self.act2 = nn.Sigmoid()
        self.hidden4 = nn.Linear(32, 16)
        self.act2 = nn.Sigmoid()
        self.hidden5 = nn.Linear(16, 8)
        self.act2 = nn.Sigmoid()
        self.hidden6 = nn.Linear(8, 4)
        self.output= nn.Linear(4, 1)




 
    def forward(self, x):
        x = self.act2(self.hidden2(x))
        x = self.act2(self.hidden3(x))
        x = self.act2(self.hidden4(x))
        x = self.act2(self.hidden5(x))
        x = self.act2(self.hidden6(x))

        return x







In [ ]:
class ModelB(nn.Module):
    def __init__(self, model):
        super(ModelB, self).__init__()

        self.input_layer = nn.Linear(8, 8)
        self.act1 = nn.Sigmoid() 
        self.model = model# Insert Model A in between input and output
        
        self.output= nn.Linear(4, 1)

        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.input_layer(x))
        x = self.model(x)  # Pass through Model A
        x = self.act_output(self.output(x))
        return x
    
    def give(self):
        return self.model

In [ ]:
def train(model,n_epochs,batch_size,traindata,lr):
    X_train=traindata[0][0]
    model=model.to(device)
    y_train=traindata[0][1]
    loss_func = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr)
    for epoch in range(n_epochs):
            for i in range(0, len(X_train), batch_size):
                Xbatch = X_train[i:i+batch_size]
                y_pred = model(Xbatch)
                ybatch = y_train[i:i+batch_size]
                loss = loss_func(y_pred, ybatch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                

            if epoch % 500 == 0 & epoch != 0:
                with torch.no_grad():
                    y_pred = model(X_train)

                accuracy = (y_pred.round() == y_train).float().mean()
                with torch.no_grad():
                    y_pred = model(X_train)



                print(f"Accuracy {accuracy}")
                print(f'Finished epoch {epoch}, latest loss {loss}')


def test(model, testloader):
    """Evaluate the network on the entire test set."""
    model=model.to(device)
    X_test=testloader[0][0]
    
    y_test=testloader[0][1]
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    loss_func = nn.MSELoss()
    with torch.no_grad():
        y_pred = model(X_test)
    accuracy = (y_pred.round() == y_test).float().mean()

    loss = loss_func(y_pred, y_test)

    return loss.tolist(), accuracy.tolist()
    


def test_whole_data(model, x,y):
    """Evaluate the network on the entire test set."""
    model=model.to(device)
    X_test=x
    y_test=y
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    loss_func = nn.MSELoss()
    with torch.no_grad():
        y_pred = model(X_test)
    accuracy = (y_pred.round() == y_test).float().mean()

    loss = loss_func(y_pred, y_test)

    return loss.tolist(), accuracy.tolist()

In [ ]:
def num_model(n):
    model_list=[]
    for i in range(n):
        model1=ModelA()


        model2=ModelB(model1)
     
        model_list.append(model2)
    return model_list


In [ ]:
def train_global_standard(n,k,n_epochs,batch_size):

    mod_list=num_model(n)
    trainloaders,testloaders,x_test,y_test=creat_data(n)

    model1=ModelA()


    glob_model=ModelB(model1)
    itterate=True
    acc=[]
    epo=[]
    curr=0
    best_avg_acc=0
    
    for i in range(k):

        for i in range(n):
            train(mod_list[i],n_epochs,batch_size,trainloaders[i],lr=0.001)

        epo.append(curr)
        curr=curr+1
        print(curr)

        res_acc=[]  
        res_loss=[]        
        for i in range(n):
            hold=test_whole_data(mod_list[i],x_test,y_test)
            print(hold)
            res_acc.append(hold[1])
            res_loss.append(hold[0])


        avg_value=sum(res_acc)/n
        avg_loss=sum(res_loss)/n
        print(f'avrage value {avg_value}')


        if avg_value > best_avg_acc:
            best_avg_acc = avg_value
        acc.append([avg_value,avg_loss])  

        parameter_lists=[]
        for model in mod_list:
            hold=model.give()
            parameter_lists.append(hold.parameters())

        



        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

        glob_hold = ModelA() 

        
        updated_params = []

        
        for glob_param, avg_param in zip(glob_hold.parameters(), avg_params):
            updated_params.append(avg_param)

       
        for glob_param, updated_param in zip(glob_hold.parameters(), updated_params):
            glob_param.data.copy_(updated_param)

        

        for model_instance in mod_list:
            model_instance.model = glob_hold

        






   
    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.title("Convergence of Federated Learning with Five Models (5 Epochs, Batch Size 32)")
    plt.legend()
    plt.axes([1, 0.4, 0.2, 0.2])
    plt.boxplot(res_acc)
    plt.xlabel('Accuracy')
    plt.title('Boxplot of Accuracy')
    plt.show()

    return best_avg_acc
    






In [ ]:
train_global_standard(5,5000,5,32)

In [14]:
def train_global_Imp(n,k,n_epochs,batch_size):

    trainloaders,testloaders,x_test,y_test=creat_data(n)
    


    
    

    mod_list=num_model(n)
    for i in range(n):
        train(mod_list[i],1000,batch_size,trainloaders[i],lr=0.001)
    
        


    

    


    best_avg_acc=0
    acc=[]
    epo=[]
    curr=0
    for i in range(k):

        for i in range(n):
            train(mod_list[i],n_epochs,batch_size,trainloaders[i],lr=0.001)

        epo.append(curr)
        curr=curr+1
        print(curr)

        res_acc=[]  
        res_loss=[]        
        for i in range(n):
            hold=test_whole_data(mod_list[i],x_test,y_test)
            print(hold)
            res_acc.append(hold[1])
            res_loss.append(hold[0])
            


        avg_value=sum(res_acc)/n
        avg_loss=sum(res_loss)/n
        print(f'avrage value {avg_value}')

        acc.append([avg_value,avg_loss])    
        if avg_value > best_avg_acc:
            best_avg_acc = avg_value


        parameter_lists=[]
        k=0
        for model in mod_list:
            if res_acc[k]>avg_value:
                print("fedder")
                hold=model.give()
                parameter_lists.append(hold.parameters())
            k=k+1





        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

        

        glob_hold = ModelA() 

        
        updated_params = []

        
        for glob_param, avg_param in zip(glob_hold.parameters(), avg_params):
            updated_params.append(avg_param)

       
        for glob_param, updated_param in zip(glob_hold.parameters(), updated_params):
            glob_param.data.copy_(updated_param)

        # Freeze all layers in glob_hold
        if curr % 50 == 0:
            print("fixer")
            
            for param in glob_hold.parameters():
                param.requires_grad = False
        

            for model_instance in mod_list:
                model_instance.model = glob_hold
                

      
            k=0
            for i in range(n):
                if res_acc[k]<avg_value:
                    print("fed_fix")
                    train(mod_list[i],n_epochs+500,batch_size,trainloaders[i],lr=0.002)
                k=k+1

            print(test_whole_data(mod_list[0],x_test,y_test))
        else:
            for model_instance in mod_list:
                model_instance.model = glob_hold




        

         
            







        



    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.title("Convergence of Federated Learning with Five Models (5 Epochs, Batch Size 32)")
    plt.legend()
    plt.axes([1, 0.4, 0.2, 0.2])
    plt.boxplot(res_acc)
    plt.xlabel('Accuracy')
    plt.title('Boxplot of Accuracy')
    plt.show()

    return best_avg_acc






In [ ]:
train_global_Imp(5,5000,5,32)


In [15]:
def creat_data(n):
    data= pd.read_csv("Heart Attack.csv")
    data_start= pd.read_csv("Heart Attack.csv")
    df_chunks = np.array_split(data_start, n)
    

    trainloaders=[]
    testloaders = []

        
    X = data.drop(columns='class')
    y = data['class']

    X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.1, random_state=0)

    X_train1 = torch.Tensor(X_train1.values)
    y_train1 = torch.Tensor(y_train1.values)
    X_train1 = torch.tensor(X_train1, dtype=torch.float32)
    y_train1= torch.tensor(y_train1, dtype=torch.float32).reshape(-1, 1)

    X_train1=X_train1.to(device)
    y_train1=y_train1.to(device)

    X_test1 = torch.Tensor(X_test1.values)
    y_test1= torch.Tensor(y_test1.values)
    X_test1 = torch.tensor(X_test1, dtype=torch.float32)
    y_test1 = torch.tensor(y_test1, dtype=torch.float32).reshape(-1, 1)

    X_test1=X_test1.to(device)
    y_test1=y_test1.to(device)
        


    for data in df_chunks:
        hold1=[]
        hold2=[]
        
        X = data.drop(columns='class')
        y = data['class']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

        X_train = torch.Tensor(X_train.values)
        y_train = torch.Tensor(y_train.values)
        X_train = torch.tensor(X_train, dtype=torch.float32)
        y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)

        X_train=X_train.to(device)
        y_train=y_train.to(device)

        X_test = torch.Tensor(X_test.values)
        y_test= torch.Tensor(y_test.values)
        X_test = torch.tensor(X_test, dtype=torch.float32)
        y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

        X_test=X_test.to(device)
        y_test=y_test.to(device)
        
        hold1.append((X_train,y_train))
        hold2.append((X_test,y_test))
        trainloaders.append(hold1)
        testloaders.append(hold2)

    return trainloaders,testloaders,X_train1,y_train1

In [16]:
# model definition
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 8)
        self.act1 = nn.Sigmoid()        
        self.hidden2 = nn.Linear(8, 16)
        self.act2 = nn.Sigmoid()
        self.hidden3 = nn.Linear(16, 32)
        self.act2 = nn.Sigmoid()
        self.hidden4 = nn.Linear(32, 16)
        self.act2 = nn.Sigmoid()
        self.hidden5 = nn.Linear(16, 8)
        self.act2 = nn.Sigmoid()
        self.hidden6 = nn.Linear(8, 4)
        self.output= nn.Linear(4, 1)
        self.act_output = nn.Sigmoid()

 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act2(self.hidden3(x))
        x = self.act2(self.hidden4(x))
        x = self.act2(self.hidden5(x))
        x = self.act2(self.hidden6(x))
        x = self.act_output(self.output(x))
        return x


In [18]:
def num_model(n):
    model_list=[]
    for i in range(n):
        model_list.append(Model())
    return model_list

In [21]:
def train_fed(n,k,n_epochs,batch_size):

    mod_list=num_model(n)
    trainloaders,testloaders,x_test,y_test=creat_data(n)


    glob_model=Model()
    acc=[]
    epo=[]
    curr=0
    best_value=0
    for i in range(k):

        for i in range(n):
            train(mod_list[i],n_epochs,batch_size,trainloaders[i],0.001)

        res_acc=[]  
        res_loss=[]        
        for i in range(n):
            hold=test_whole_data(mod_list[i],x_test,y_test)

            res_acc.append(hold[0])
            res_loss.append(hold[1])
        
    

        parameter_lists = [model.parameters() for model in mod_list]
        avg_acc=sum(res_acc)/n
        if avg_acc>best_value:
            best_value=avg_acc

        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

        for glob_param, param in zip(glob_model.parameters(), avg_params):
            glob_param.data.copy_(param)

        

        mod_list=[]

        for i in range(n):
            mod_list.append(glob_model)


        epo.append(curr)
        curr=+curr+1
        print(curr)
        acc.append(test_whole_data(mod_list[i],x_test,y_test))
    




        




    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.title("Convergence of Federated Learning with Five Models (5 Epochs, Batch Size 32)")
    plt.legend()
    plt.axes([1, 0.4, 0.2, 0.2])
    plt.boxplot(res_acc)
    plt.xlabel('Accuracy')
    plt.title('Boxplot of Accuracy')
    plt.show()

    return best_value

In [22]:
train_fed(5,1000,5,32)

c:\Users\isak1\miniconda3\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\isak1\AppData\Local\Temp\ipykernel_20268\2264630936.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train1 = torch.tensor(X_train1, dtype=torch.float32)
C:\Users\isak1\AppData\Local\Temp\ipykernel_20268\2264630936.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train1= torch.tensor(y_train1, dtype=torch.float32).reshape(-1, 1)
C:\Users\isak1\AppData\Local\Temp\ipykernel_20268\2264630936.py:26: UserWarning: To copy construct from a t

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


KeyboardInterrupt: 